<a href="https://colab.research.google.com/github/vaegev/tensorflow_basics/blob/master/ann_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load Data

In [0]:
import tensorflow as tf
import numpy as np

(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

11493376/11490434 [==============================] - 0s 0us/step


# Estimator High-Level API

In [0]:
feature_cols = [tf.feature_column.numeric_column("X", shape=[28*28])]
dnn_clf = tf.estimator.DNNClassifier(hidden_units = [300, 100], 
                           feature_columns = feature_cols, 
                           n_classes = 10)
input_fn = tf.estimator.inputs.numpy_input_fn(x={"X": X_train}, y=y_train, 
                                              batch_size=50, 
                                              num_epochs=40, 
                                              shuffle=True)
dnn_clf.train(input_fn=input_fn)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmphi5y5uvz', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f46a9d49b38>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, u

In [0]:
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"X": X_test}, y=y_test, shuffle=False)
eval_results = dnn_clf.evaluate(input_fn=test_input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-01-26-15:06:14
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmphi5y5uvz/model.ckpt-44000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-01-26-15:06:14
INFO:tensorflow:Saving dict for global step 44000: accuracy = 0.9802, average_loss = 0.10477246, global_step = 44000, loss = 13.262337
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 44000: /tmp/tmphi5y5uvz/model.ckpt-44000


In [0]:
eval_results

{'accuracy': 0.9802,
 'average_loss': 0.10477246,
 'global_step': 44000,
 'loss': 13.262337}

In [0]:
y_pred_iter = dnn_clf.predict(input_fn=test_input_fn)
y_pred = list(y_pred_iter)
y_pred[0]

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmphi5y5uvz/model.ckpt-44000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


{'class_ids': array([7]),
 'classes': array([b'7'], dtype=object),
 'logits': array([ -6.6532807,   1.4519783,   5.54506  ,   7.3256545,  -8.448579 ,
         -8.443406 , -25.096392 ,  30.122057 ,  -1.9050634,   9.538804 ],
       dtype=float32),
 'probabilities': array([1.06825274e-16, 3.53788125e-13, 2.12005118e-11, 1.25790670e-10,
        1.77412970e-17, 1.78333043e-17, 1.04455794e-24, 1.00000000e+00,
        1.23253458e-14, 1.15028986e-09], dtype=float32)}

# Using plain TensorFlow